# Import Dependencies

In [1]:
# Dependencies and Setup FOR API
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import requests
import time
#from datetime import datetime
import datetime as dt
import json

# Import API key
from api_keys import tmdb_api_key


# Extract CSV Files

### Store CSV into DataFrame

In [2]:
# Store Disney file into dataframe 
disney_csv_file = "./Datasets/Disney_titles.csv"
disney_titles_df = pd.read_csv(disney_csv_file)
disney_titles_df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,tm74391,Fantasia,MOVIE,Walt Disney's timeless masterpiece is an extra...,1940,G,120,"['animation', 'family', 'music', 'fantasy']",['US'],NaN,tt0032455,7.7,94681.0,57.751,7.4
1,tm67803,Snow White and the Seven Dwarfs,MOVIE,"A beautiful girl, Snow White, takes refuge in ...",1937,G,83,"['fantasy', 'family', 'romance', 'animation', ...",['US'],NaN,tt0029583,7.6,195321.0,107.137,7.1
2,tm82546,Pinocchio,MOVIE,Lonely toymaker Geppetto has his wishes answer...,1940,G,88,"['animation', 'comedy', 'family', 'fantasy']",['US'],NaN,tt0032910,7.5,141937.0,71.160,7.1
3,tm79357,Bambi,MOVIE,Bambi's tale unfolds from season to season as ...,1942,G,70,"['animation', 'drama', 'family']",['US'],NaN,tt0034492,7.3,140406.0,68.136,7.0
4,tm62671,Treasure Island,MOVIE,Enchanted by the idea of locating treasure bur...,1950,PG,96,"['family', 'action']","['GB', 'US']",NaN,tt0043067,6.9,8229.0,10.698,6.5


In [4]:
# Store Netflix file into dataframe 
netflix_csv_file = "./Datasets/Netflix_titles.csv"
netflix_titles_df = pd.read_csv(netflix_csv_file)
netflix_titles_df.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


# Transform Disney and Netflix Dataframes

### Create New Dataframe with just the columns we want for Disney

In [5]:
new_disney_df = disney_titles_df[["title", "description", "release_year", "tmdb_popularity", "tmdb_score", "type"]]
new_disney_df.head()

,title,description,release_year,tmdb_popularity,tmdb_score,type
0,Fantasia,Walt Disney's timeless masterpiece is an extra...,1940,57.751,7.4,MOVIE
1,Snow White and the Seven Dwarfs,"A beautiful girl, Snow White, takes refuge in ...",1937,107.137,7.1,MOVIE
2,Pinocchio,Lonely toymaker Geppetto has his wishes answer...,1940,71.160,7.1,MOVIE
3,Bambi,Bambi's tale unfolds from season to season as ...,1942,68.136,7.0,MOVIE
4,Treasure Island,Enchanted by the idea of locating treasure bur...,1950,10.698,6.5,MOVIE


### Create New Dataframe with just the columns we want for Netflix

In [6]:
new_netflix_df = netflix_titles_df[["title", "description", "release_year", "tmdb_popularity", "tmdb_score", "type"]]
new_netflix_df.head()

,title,description,release_year,tmdb_popularity,tmdb_score,type
0,Five Came Back: The Reference Films,This collection includes 12 World War II-era p...,1945,0.600,NaN,SHOW
1,Taxi Driver,A mentally unstable Vietnam War veteran works ...,1976,40.965,8.179,MOVIE
2,Deliverance,Intent on seeing the Cahulawassee River before...,1972,10.010,7.300,MOVIE
3,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",1975,15.461,7.811,MOVIE
4,The Dirty Dozen,12 American military prisoners in World War II...,1967,20.398,7.600,MOVIE


### Combine Disney and Netflix Dataframes into one

In [7]:
frames = [new_disney_df, new_netflix_df]
combined_df = pd.concat(frames)
combined_df.head()

,title,description,release_year,tmdb_popularity,tmdb_score,type
0,Fantasia,Walt Disney's timeless masterpiece is an extra...,1940,57.751,7.4,MOVIE
1,Snow White and the Seven Dwarfs,"A beautiful girl, Snow White, takes refuge in ...",1937,107.137,7.1,MOVIE
2,Pinocchio,Lonely toymaker Geppetto has his wishes answer...,1940,71.160,7.1,MOVIE
3,Bambi,Bambi's tale unfolds from season to season as ...,1942,68.136,7.0,MOVIE
4,Treasure Island,Enchanted by the idea of locating treasure bur...,1950,10.698,6.5,MOVIE


# Extract JSON Files 

### Perform API call for TMDB top rated movies

In [8]:
#Create list to store responses
response_list = []

x = range(1,25,1)
### Perform API Call
for page in x:
    print(f"Performing call number: {page}")
    url=f"https://api.themoviedb.org/3/movie/top_rated?api_key={tmdb_api_key}&language=en-US&page={page}"
    response = requests.get(url)
    response_list.append(response.json())


Performing call number: 1
Performing call number: 2
Performing call number: 3
Performing call number: 4
Performing call number: 5
Performing call number: 6
Performing call number: 7
Performing call number: 8
Performing call number: 9
Performing call number: 10
Performing call number: 11
Performing call number: 12
Performing call number: 13
Performing call number: 14
Performing call number: 15
Performing call number: 16
Performing call number: 17
Performing call number: 18
Performing call number: 19
Performing call number: 20
Performing call number: 21
Performing call number: 22
Performing call number: 23
Performing call number: 24


### Perform API call for TMDB top rated TV shows

In [9]:
# Create empty list to store responses
tv_response_list = []

y = range(1,25,1)
### Perform API Call
for page in y:
    print(f"Performing call number: {page}")
    url=f"https://api.themoviedb.org/3/tv/top_rated?api_key={tmdb_api_key}&language=en-US&page={page}"
    response = requests.get(url)
    tv_response_list.append(response.json())
    

Performing call number: 1
Performing call number: 2
Performing call number: 3
Performing call number: 4
Performing call number: 5
Performing call number: 6
Performing call number: 7
Performing call number: 8
Performing call number: 9
Performing call number: 10
Performing call number: 11
Performing call number: 12
Performing call number: 13
Performing call number: 14
Performing call number: 15
Performing call number: 16
Performing call number: 17
Performing call number: 18
Performing call number: 19
Performing call number: 20
Performing call number: 21
Performing call number: 22
Performing call number: 23
Performing call number: 24


### Put results of API calls into Dataframes

In [11]:
# Create empty list to store 'results' for TMDB movies
tmdb_movies_list = []

for x in response_list:
    for_each_page = [p for p in x["results"]]
    tmdb_movies_list.extend(for_each_page)

tmdb_movies_df = pd.DataFrame.from_records(tmdb_movies_list)
tmdb_movies_df.head()  

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,"[18, 80]",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",98.808,/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,1972-03-14,The Godfather,False,8.7,17023
1,False,/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg,"[18, 80]",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,87.931,/hBcY0fE9pfXzvVaY4GKarweriG2.jpg,1994-09-23,The Shawshank Redemption,False,8.7,22777
2,False,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,"[18, 80]",240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,59.234,/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,1974-12-20,The Godfather Part II,False,8.6,10316
3,False,/rl7Jw8PjhSIjArOlDNv0JQPL1ZV.jpg,"[10749, 18]",851644,ko,20세기 소녀,Yeon-du asks her best friend Bora to collect a...,174.771,/od22ftNnyag0TTxcnJhlsu3aLoU.jpg,2022-10-06,20th Century Girl,False,8.6,312
4,False,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,"[18, 36, 10752]",424,en,Schindler's List,The true story of how businessman Oskar Schind...,54.271,/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,1993-12-15,Schindler's List,False,8.6,13513


In [13]:
# Create empty list to store 'results' for TMDB TV shows
tmdb_tv_list = []

for y in tv_response_list:
    for_each_tv_page = [p for p in y["results"]]
    tmdb_tv_list.extend(for_each_tv_page)

tmdb_tv_df = pd.DataFrame.from_records(tmdb_tv_list)
tmdb_tv_df.head()  

,backdrop_path,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,overview,popularity,poster_path,vote_average,vote_count
0,/7q448EVOnuE3gVAx24krzO7SNXM.jpg,2021-09-03,[10764],130392,The D'Amelio Show,[US],en,The D'Amelio Show,From relative obscurity and a seemingly normal...,25.358,/z0iCS5Znx7TeRwlYSd4c01Z0lFx.jpg,9.0,3146
1,/84XPpjGvxNyExjSuLQe0SzioErt.jpg,2008-01-20,[18],1396,Breaking Bad,[US],en,Breaking Bad,"When Walter White, a New Mexico chemistry teac...",320.577,/ggFHVNu6YYI5L9pCfOacjizRGt.jpg,8.8,10493
2,/8gsbbKCDI3CrQg0UExFoUjRUmHM.jpg,2022-11-23,"[10765, 9648, 35]",119051,Wednesday,[US],en,Wednesday,"Wednesday Addams is sent to Nevermore Academy,...",8669.948,/9PFonBhy4cQy7Jz20NpMygczOkv.jpg,8.8,2108
3,/rkB4LyZHo1NHXFEDHl9vSD9r1lI.jpg,2021-11-06,"[16, 10765, 10759, 18]",94605,Arcane,[US],en,Arcane,Amid the stark discord of twin cities Piltover...,79.198,/fqldf2t8ztc9aiwn3k6mlX3tvRT.jpg,8.8,2727
4,/uGy4DCmM33I7l86W7iCskNkvmLD.jpg,2013-12-02,"[16, 35, 10765, 10759]",60625,Rick and Morty,[US],en,Rick and Morty,Rick is a mentally-unbalanced but scientifical...,911.371,/cvhNj9eoRBe5SxjCbQTkh05UP5K.jpg,8.7,7482


# Transform TMDB Dataframes

### Rename columns in TMDB Movie Dataframe to match Disney/ Netflix dataframe

In [14]:
tmdb_movies_df.rename(columns={'popularity':'tmdb_popularity', 'overview':'description', 'vote_average':'tmdb_score', 'release_date':'release_year' }, inplace = True)
tmdb_movies_df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,description,tmdb_popularity,poster_path,release_year,title,video,tmdb_score,vote_count
0,False,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,"[18, 80]",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",98.808,/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,1972-03-14,The Godfather,False,8.7,17023
1,False,/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg,"[18, 80]",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,87.931,/hBcY0fE9pfXzvVaY4GKarweriG2.jpg,1994-09-23,The Shawshank Redemption,False,8.7,22777
2,False,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,"[18, 80]",240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,59.234,/hek3koDUyRQk7FIhPXsa6mT2Zc3.jpg,1974-12-20,The Godfather Part II,False,8.6,10316
3,False,/rl7Jw8PjhSIjArOlDNv0JQPL1ZV.jpg,"[10749, 18]",851644,ko,20세기 소녀,Yeon-du asks her best friend Bora to collect a...,174.771,/od22ftNnyag0TTxcnJhlsu3aLoU.jpg,2022-10-06,20th Century Girl,False,8.6,312
4,False,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,"[18, 36, 10752]",424,en,Schindler's List,The true story of how businessman Oskar Schind...,54.271,/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,1993-12-15,Schindler's List,False,8.6,13513


### Rename columns in TMDB TV Dataframe to match Disney/ Netflix dataframe

In [15]:
tmdb_tv_df.rename(columns={'name':'title', 'popularity':'tmdb_popularity', 'overview':'description', 'vote_average':'tmdb_score', 'first_air_date':'release_year' }, inplace = True)
tmdb_tv_df.head()

,backdrop_path,release_year,genre_ids,id,title,origin_country,original_language,original_name,description,tmdb_popularity,poster_path,tmdb_score,vote_count
0,/7q448EVOnuE3gVAx24krzO7SNXM.jpg,2021-09-03,[10764],130392,The D'Amelio Show,[US],en,The D'Amelio Show,From relative obscurity and a seemingly normal...,25.358,/z0iCS5Znx7TeRwlYSd4c01Z0lFx.jpg,9.0,3146
1,/84XPpjGvxNyExjSuLQe0SzioErt.jpg,2008-01-20,[18],1396,Breaking Bad,[US],en,Breaking Bad,"When Walter White, a New Mexico chemistry teac...",320.577,/ggFHVNu6YYI5L9pCfOacjizRGt.jpg,8.8,10493
2,/8gsbbKCDI3CrQg0UExFoUjRUmHM.jpg,2022-11-23,"[10765, 9648, 35]",119051,Wednesday,[US],en,Wednesday,"Wednesday Addams is sent to Nevermore Academy,...",8669.948,/9PFonBhy4cQy7Jz20NpMygczOkv.jpg,8.8,2108
3,/rkB4LyZHo1NHXFEDHl9vSD9r1lI.jpg,2021-11-06,"[16, 10765, 10759, 18]",94605,Arcane,[US],en,Arcane,Amid the stark discord of twin cities Piltover...,79.198,/fqldf2t8ztc9aiwn3k6mlX3tvRT.jpg,8.8,2727
4,/uGy4DCmM33I7l86W7iCskNkvmLD.jpg,2013-12-02,"[16, 35, 10765, 10759]",60625,Rick and Morty,[US],en,Rick and Morty,Rick is a mentally-unbalanced but scientifical...,911.371,/cvhNj9eoRBe5SxjCbQTkh05UP5K.jpg,8.7,7482


### Make New TMDB Movies Dataframe containing only the columns we need

In [16]:
tmdb_movies_cleaned = tmdb_movies_df[['title', 'description', 'release_year', 'tmdb_popularity', 'tmdb_score']]
tmdb_movies_cleaned.head()

,title,description,release_year,tmdb_popularity,tmdb_score
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,98.808,8.7
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,87.931,8.7
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,59.234,8.6
3,20th Century Girl,Yeon-du asks her best friend Bora to collect a...,2022-10-06,174.771,8.6
4,Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,54.271,8.6


### Make New TMDB TV Dataframe containing only the columns we need

In [17]:
tmdb_tv_cleaned = tmdb_tv_df[['title', 'description', 'release_year', 'tmdb_popularity', 'tmdb_score']]
tmdb_tv_cleaned.head()

,title,description,release_year,tmdb_popularity,tmdb_score
0,The D'Amelio Show,From relative obscurity and a seemingly normal...,2021-09-03,25.358,9.0
1,Breaking Bad,"When Walter White, a New Mexico chemistry teac...",2008-01-20,320.577,8.8
2,Wednesday,"Wednesday Addams is sent to Nevermore Academy,...",2022-11-23,8669.948,8.8
3,Arcane,Amid the stark discord of twin cities Piltover...,2021-11-06,79.198,8.8
4,Rick and Morty,Rick is a mentally-unbalanced but scientifical...,2013-12-02,911.371,8.7


### Add Type column to cleaned movies database with type as 'MOVIE'

In [18]:
tmdb_movies_cleaned['type']="MOVIE"
tmdb_movies_cleaned.head()

C:\AnacondaPython\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,title,description,release_year,tmdb_popularity,tmdb_score,type
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,98.808,8.7,MOVIE
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,87.931,8.7,MOVIE
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,59.234,8.6,MOVIE
3,20th Century Girl,Yeon-du asks her best friend Bora to collect a...,2022-10-06,174.771,8.6,MOVIE
4,Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,54.271,8.6,MOVIE


### Add Type column to cleaned TV database with type as 'SHOW'

In [19]:
tmdb_tv_cleaned['type']="SHOW"
tmdb_tv_cleaned

C:\AnacondaPython\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,title,description,release_year,tmdb_popularity,tmdb_score,type
0,The D'Amelio Show,From relative obscurity and a seemingly normal...,2021-09-03,25.358,9.0,SHOW
1,Breaking Bad,"When Walter White, a New Mexico chemistry teac...",2008-01-20,320.577,8.8,SHOW
2,Wednesday,"Wednesday Addams is sent to Nevermore Academy,...",2022-11-23,8669.948,8.8,SHOW
3,Arcane,Amid the stark discord of twin cities Piltover...,2021-11-06,79.198,8.8,SHOW
4,Rick and Morty,Rick is a mentally-unbalanced but scientifical...,2013-12-02,911.371,8.7,SHOW
...,...,...,...,...,...,...
475,The Crown,"The gripping, decades-spanning inside story of...",2016-11-04,163.000,8.3,SHOW
476,Wizards: Tales of Arcadia,Merlin’s apprentice joins Arcadia’s heroes on ...,2020-08-07,25.639,8.3,SHOW
477,The Rose of Versailles,"Raised from birth as a man, the Lady Oscar com...",1979-10-10,13.331,8.3,SHOW
478,Blood+,"Unable to remember the past, high school senio...",2005-10-08,20.719,8.3,SHOW


### Combine cleaned Movies and TV Dataframes into one Dataframe with reset index

In [20]:
tmdb_frames = [tmdb_movies_cleaned, tmdb_tv_cleaned]
combined_tmdb_df = pd.concat(tmdb_frames, ignore_index=True)
combined_tmdb_df 

,title,description,release_year,tmdb_popularity,tmdb_score,type
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972-03-14,98.808,8.7,MOVIE
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994-09-23,87.931,8.7,MOVIE
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974-12-20,59.234,8.6,MOVIE
3,20th Century Girl,Yeon-du asks her best friend Bora to collect a...,2022-10-06,174.771,8.6,MOVIE
4,Schindler's List,The true story of how businessman Oskar Schind...,1993-12-15,54.271,8.6,MOVIE
...,...,...,...,...,...,...
955,The Crown,"The gripping, decades-spanning inside story of...",2016-11-04,163.000,8.3,SHOW
956,Wizards: Tales of Arcadia,Merlin’s apprentice joins Arcadia’s heroes on ...,2020-08-07,25.639,8.3,SHOW
957,The Rose of Versailles,"Raised from birth as a man, the Lady Oscar com...",1979-10-10,13.331,8.3,SHOW
958,Blood+,"Unable to remember the past, high school senio...",2005-10-08,20.719,8.3,SHOW


### Find any null values in the 'release_year' column and drop them

In [26]:
# Drop NaN values
combined_tmdb_df = combined_tmdb_df.dropna(subset = ['release_year'])                                            

# reset index 
combined_tmdb_df.reset_index(drop=True)

,title,description,release_year,tmdb_popularity,tmdb_score,type
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1972.0,98.808,8.7,MOVIE
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1994.0,87.931,8.7,MOVIE
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,1974.0,59.234,8.6,MOVIE
3,20th Century Girl,Yeon-du asks her best friend Bora to collect a...,2022.0,174.771,8.6,MOVIE
4,Schindler's List,The true story of how businessman Oskar Schind...,1993.0,54.271,8.6,MOVIE
...,...,...,...,...,...,...
954,The Crown,"The gripping, decades-spanning inside story of...",2016.0,163.000,8.3,SHOW
955,Wizards: Tales of Arcadia,Merlin’s apprentice joins Arcadia’s heroes on ...,2020.0,25.639,8.3,SHOW
956,The Rose of Versailles,"Raised from birth as a man, the Lady Oscar com...",1979.0,13.331,8.3,SHOW
957,Blood+,"Unable to remember the past, high school senio...",2005.0,20.719,8.3,SHOW


### Format 'release_year' column to match Disney/ Netflix Dataframe format

In [27]:
# Convert 'release_year' to datetime type
combined_tmdb_df['release_year'] = pd.to_datetime(combined_tmdb_df['release_year'])

# Drop month and day from the date
combined_tmdb_df['release_year'] = combined_tmdb_df['release_year'].dt.year
combined_tmdb_df

C:\AnacondaPython\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\AnacondaPython\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,title,description,release_year,tmdb_popularity,tmdb_score,type
0,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",1970,98.808,8.7,MOVIE
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,1970,87.931,8.7,MOVIE
2,The Godfather Part II,In the continuing saga of the Corleone crime f...,1970,59.234,8.6,MOVIE
3,20th Century Girl,Yeon-du asks her best friend Bora to collect a...,1970,174.771,8.6,MOVIE
4,Schindler's List,The true story of how businessman Oskar Schind...,1970,54.271,8.6,MOVIE
...,...,...,...,...,...,...
955,The Crown,"The gripping, decades-spanning inside story of...",1970,163.000,8.3,SHOW
956,Wizards: Tales of Arcadia,Merlin’s apprentice joins Arcadia’s heroes on ...,1970,25.639,8.3,SHOW
957,The Rose of Versailles,"Raised from birth as a man, the Lady Oscar com...",1970,13.331,8.3,SHOW
958,Blood+,"Unable to remember the past, high school senio...",1970,20.719,8.3,SHOW


# Transform the Final Dataframe

### Combine Disney/ Netflix and TMDB Movies/ TV Dataframes into Final Dataframe with reset index

In [28]:
popular_entertainment_frames = [combined_df, combined_tmdb_df]
popular_entertainment_df = pd.concat(popular_entertainment_frames, ignore_index=True)
popular_entertainment_df

,title,description,release_year,tmdb_popularity,tmdb_score,type
0,Fantasia,Walt Disney's timeless masterpiece is an extra...,1940,57.751,7.4,MOVIE
1,Snow White and the Seven Dwarfs,"A beautiful girl, Snow White, takes refuge in ...",1937,107.137,7.1,MOVIE
2,Pinocchio,Lonely toymaker Geppetto has his wishes answer...,1940,71.160,7.1,MOVIE
3,Bambi,Bambi's tale unfolds from season to season as ...,1942,68.136,7.0,MOVIE
4,Treasure Island,Enchanted by the idea of locating treasure bur...,1950,10.698,6.5,MOVIE
...,...,...,...,...,...,...
8339,The Crown,"The gripping, decades-spanning inside story of...",1970,163.000,8.3,SHOW
8340,Wizards: Tales of Arcadia,Merlin’s apprentice joins Arcadia’s heroes on ...,1970,25.639,8.3,SHOW
8341,The Rose of Versailles,"Raised from birth as a man, the Lady Oscar com...",1970,13.331,8.3,SHOW
8342,Blood+,"Unable to remember the past, high school senio...",1970,20.719,8.3,SHOW


### Drop rows with NaN value for 'tmdb_popularity' column

In [29]:
# Drop NaN values
popular_entertainment_df = popular_entertainment_df.dropna(subset = ['tmdb_popularity'])                                            

# reset index 
popular_entertainment_df.reset_index(drop=True)


,title,description,release_year,tmdb_popularity,tmdb_score,type
0,Fantasia,Walt Disney's timeless masterpiece is an extra...,1940,57.751,7.4,MOVIE
1,Snow White and the Seven Dwarfs,"A beautiful girl, Snow White, takes refuge in ...",1937,107.137,7.1,MOVIE
2,Pinocchio,Lonely toymaker Geppetto has his wishes answer...,1940,71.160,7.1,MOVIE
3,Bambi,Bambi's tale unfolds from season to season as ...,1942,68.136,7.0,MOVIE
4,Treasure Island,Enchanted by the idea of locating treasure bur...,1950,10.698,6.5,MOVIE
...,...,...,...,...,...,...
8237,The Crown,"The gripping, decades-spanning inside story of...",1970,163.000,8.3,SHOW
8238,Wizards: Tales of Arcadia,Merlin’s apprentice joins Arcadia’s heroes on ...,1970,25.639,8.3,SHOW
8239,The Rose of Versailles,"Raised from birth as a man, the Lady Oscar com...",1970,13.331,8.3,SHOW
8240,Blood+,"Unable to remember the past, high school senio...",1970,20.719,8.3,SHOW


### Drop rows with duplicate title

In [30]:
# Drop duplicate title rows
popular_entertainment_df = popular_entertainment_df.drop_duplicates(subset='title', keep="first")

# reset index 
popular_entertainment_df.reset_index(drop=True)

,title,description,release_year,tmdb_popularity,tmdb_score,type
0,Fantasia,Walt Disney's timeless masterpiece is an extra...,1940,57.751,7.4,MOVIE
1,Snow White and the Seven Dwarfs,"A beautiful girl, Snow White, takes refuge in ...",1937,107.137,7.1,MOVIE
2,Pinocchio,Lonely toymaker Geppetto has his wishes answer...,1940,71.160,7.1,MOVIE
3,Bambi,Bambi's tale unfolds from season to season as ...,1942,68.136,7.0,MOVIE
4,Treasure Island,Enchanted by the idea of locating treasure bur...,1950,10.698,6.5,MOVIE
...,...,...,...,...,...,...
7901,From,Unravel the mystery of a nightmarish town in m...,1970,144.977,8.3,SHOW
7902,The Handmaid's Tale,"Set in a dystopian future, a woman is forced t...",1970,132.851,8.3,SHOW
7903,The Rose of Versailles,"Raised from birth as a man, the Lady Oscar com...",1970,13.331,8.3,SHOW
7904,Blood+,"Unable to remember the past, high school senio...",1970,20.719,8.3,SHOW


### Sort dataframe in descending order by TMDB popularity; reset index

In [31]:
# Sorting in descending order
popular_entertainment_df = popular_entertainment_df.sort_values('tmdb_popularity', ascending=False)

# resetting index 
popular_entertainment_df.reset_index(drop=True)

,title,description,release_year,tmdb_popularity,tmdb_score,type
0,Moon Knight,"When Steven Grant, a mild-mannered gift-shop e...",2022,9323.832000,8.5,SHOW
1,Wednesday,"Wednesday Addams is sent to Nevermore Academy,...",1970,8669.948000,8.8,SHOW
2,The Woman King,"The story of the Agojie, the all-female unit o...",1970,3396.412000,7.9,MOVIE
3,Turning Red,Thirteen-year-old Mei is experiencing the awkw...,2022,2331.811000,7.4,MOVIE
4,Doctor Strange,"After his career is destroyed, a brilliant but...",2016,2286.312000,7.4,MOVIE
...,...,...,...,...,...,...
7901,Five Came Back: The Reference Films,This collection includes 12 World War II-era p...,1945,0.600000,NaN,SHOW
7902,Beyblade Burst Turbo,NaN,2018,0.600000,0.5,SHOW
7903,Roll it Back,Roll It Back is the aftershow for Just Roll Wi...,2019,0.600000,NaN,SHOW
7904,Blade Runner: The Final Cut,"Visually spectacular, intensely action-packed ...",2007,0.055091,9.0,MOVIE


# Load the Transformed Final Dataframe

### Connect to local database

In [32]:
protocol = 'postgresql'
username = 'postgres'
password = 'postgres'
host = 'localhost'
port = 5432
database_name = 'Entertainment_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

### Check for tables

In [33]:
engine.table_names()

['popular_entertainment', 'disney', 'netflix', 'Disney', 'Netflix']

### Use pandas to load csv converted DataFrame into database

In [34]:
popular_entertainment_df.to_sql(name='popular_entertainment', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the popular_entertainment table
* NOTE: can also check using pgAdmin

In [35]:
pd.read_sql_query('select * from popular_entertainment', con=engine).head()

,title,description,release_year,tmdb_popularity,tmdb_score,type
0,Moon Knight,"When Steven Grant, a mild-mannered gift-shop e...",2022,9323.832,8.5,SHOW
1,Wednesday,"Wednesday Addams is sent to Nevermore Academy,...",1970,8669.948,8.8,SHOW
2,The Woman King,"The story of the Agojie, the all-female unit o...",1970,3396.412,7.9,MOVIE
3,Turning Red,Thirteen-year-old Mei is experiencing the awkw...,2022,2331.811,7.4,MOVIE
4,Doctor Strange,"After his career is destroyed, a brilliant but...",2016,2286.312,7.4,MOVIE
